In [1]:
import asyncio
import ccxt.pro as ccxtpro

exchange = ccxtpro.bybit({
    'apiKey': 'WvQdYKShPIMXVGDPvn',            
    'secret': 'Y3Nx62HzOgzmA02RicNimnWeJNh3gH5hZkkJ',
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',
    },
})
# 在下單前，先異步載入市場 (markets) 定義
await exchange.load_markets()


{'BTC/USDT': {'id': 'BTCUSDT',
  'lowercaseId': None,
  'symbol': 'BTC/USDT',
  'base': 'BTC',
  'quote': 'USDT',
  'settle': None,
  'baseId': 'BTC',
  'quoteId': 'USDT',
  'settleId': None,
  'type': 'spot',
  'spot': True,
  'margin': True,
  'swap': False,
  'future': False,
  'option': False,
  'index': False,
  'active': True,
  'contract': False,
  'linear': None,
  'inverse': None,
  'subType': None,
  'taker': 0.001,
  'maker': 0.001,
  'contractSize': None,
  'expiry': None,
  'expiryDatetime': None,
  'strike': None,
  'optionType': None,
  'precision': {'amount': 1e-06,
   'price': 0.1,
   'cost': None,
   'base': None,
   'quote': None},
  'limits': {'leverage': {'min': 1.0, 'max': None},
   'amount': {'min': 1.1e-05, 'max': 83.0},
   'price': {'min': None, 'max': None},
   'cost': {'min': 5.0, 'max': 8000000.0}},
  'marginModes': {'cross': None, 'isolated': None},
  'created': None,
  'info': {'symbol': 'BTCUSDT',
   'baseCoin': 'BTC',
   'quoteCoin': 'USDT',
   'innovati

In [3]:
symbol   = 'XAUT/USDT'  # Bybit USDT 永續合約統一標記
leverage = 1            # 5 倍槓桿
await exchange.set_leverage(leverage, symbol+':USDT') 

{'retCode': '0',
 'retMsg': 'OK',
 'result': {},
 'retExtInfo': {},
 'time': '1748054067136'}

In [18]:
async def fetch_all_orders(exchange, symbol):
    # 并行获取三种状态的订单
    open_orders_task     = asyncio.create_task(exchange.fetchOpenOrders(symbol))
    closed_orders_task   = asyncio.create_task(exchange.fetchClosedOrders(symbol))
    canceled_orders_task = asyncio.create_task(exchange.fetchCanceledOrders(symbol))

    open_orders, closed_orders, canceled_orders = await asyncio.gather(
        open_orders_task,
        closed_orders_task,
        canceled_orders_task,
    )

    # 给每笔订单加上 source 字段，便于后续区分
    for o in open_orders:     o['__source'] = 'open'
    for o in closed_orders:   o['__source'] = 'closed'
    for o in canceled_orders: o['__source'] = 'canceled'

    # 合并返回
    return open_orders + closed_orders + canceled_orders

order_list = await fetch_all_orders(exchange,'XAUT/USDT')

In [19]:
x = await exchange.fetchOpenOrders(symbol)
x

[{'info': {'symbol': 'XAUTUSDT',
   'orderType': 'Limit',
   'orderLinkId': '1957213932062252545',
   'slLimitPrice': '0',
   'orderId': '1957213932062252544',
   'cancelType': 'UNKNOWN',
   'avgPrice': '0.0',
   'stopOrderType': '',
   'lastPriceOnCreated': '',
   'orderStatus': 'New',
   'takeProfit': '0',
   'cumExecValue': '0.000000',
   'smpType': 'None',
   'triggerDirection': '0',
   'blockTradeId': '',
   'isLeverage': '0',
   'rejectReason': 'EC_NoError',
   'price': '3000.0',
   'orderIv': '',
   'createdTime': '1748054082340',
   'tpTriggerBy': '',
   'positionIdx': '0',
   'trailingPercentage': '0',
   'timeInForce': 'GTC',
   'leavesValue': '3.000000',
   'basePrice': '3355.7',
   'updatedTime': '1748054082344',
   'side': 'Buy',
   'smpGroup': '0',
   'triggerPrice': '0.0',
   'tpLimitPrice': '0',
   'trailingValue': '0',
   'cumExecFee': '0',
   'leavesQty': '0.001',
   'slTriggerBy': '',
   'closeOnTrigger': False,
   'placeType': '',
   'cumExecQty': '0',
   'reduceOnl

In [21]:
symbol     = 'XAUT/USDT'    # Bybit USDT 永續合約的統一標記
order_type = 'limit'
side       = 'buy'
amount     = 0.001         # 合約數量 (視 Bybit 合約單位而定)
price      = 3000      # 單價
# 創建限價買單
order = await exchange.create_order(symbol, order_type, side, amount, price)
print('Order response:', order)

Order response: {'info': {'orderId': '1957215877095235072', 'orderLinkId': '1957215877095235073'}, 'id': '1957215877095235072', 'clientOrderId': '1957215877095235073', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'XAUT/USDT', 'type': None, 'timeInForce': None, 'postOnly': None, 'reduceOnly': None, 'side': None, 'price': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'fees': [], 'stopPrice': None}


In [6]:
import asyncio
import ccxt.pro as ccxtpro
symbol = 'XAUT/USDT:USDT'
exchange = ccxtpro.bybit({
    'apiKey': 'WvQdYKShPIMXVGDPvn',            
    'secret': 'Y3Nx62HzOgzmA02RicNimnWeJNh3gH5hZkkJ',
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',
    },
})
await exchange.load_markets()
# 切换到 Cross Margin（主合约钱包）
await exchange.set_margin_mode('cross', symbol)
# 再次设置杠杆（双向或单向都一致）
await exchange.set_leverage(10, symbol, { 'marginMode': 'cross' })


sell = await exchange.create_order(symbol, 'limit', 'sell', 0.002, 3500.65)
# await exchange.set_leverage(20, symbol+':USDT')
print('Exchange initialized')

Exchange initialized


In [1]:
for i in range(1):
    print(i)

0
